# RageSimulator
This simulator is written in rage against how awful the original Piccolissimo simulator is, and rewriting it. 

## Relevant Equations 

### Newton-Euler (equations 2.2 and 2.3)
$$\mathbf{\dot{v}^I_F} = (\mathbf{f_{aero}} + m \mathbf{g} - m \mathbf{\omega^{FI} }\times \mathbf{v^I_F})$$

$$\mathbf{\dot{\omega}^{FI}} = \mathbf{I}^{-1}(\mathbf{\tau_{aero}} - \mathbf{\omega^{FI} }\times \mathbf{I\omega^{BI}})$$

* $v_F^I$ is velocity of the center of mass of flyer frame in inertial/world frame
* $\mathbf{\omega^{FI} } = [p\ q\ r]^T$ is the angular velocity of flyer frame w.r.t. inertial
* $\mathbf{\omega^{BI} }$ is the angular velocity of body frame wrt inertial (same as flyer but with rotation about z axis).




### Blade Equations (equations 3.8)
$$[v_E^I]^E = \mathbf{R^E_F}[v^I_F]^F + \mathbf{R^E_F}[\omega^{SF}]^F \times \mathbf{R^E_F} [S_{EF}]^F + \mathbf{R^E_F}[\nu]^F$$
$$\gamma = arctan(\frac{[v_E^I]^E [\hat{z}]^E}{[v_E^I]^P [\hat{x}]^E})$$


* $[\nu]^F = [0\ 0\ \nu]^T$ is induced inflow velocity
* $\nu$ is the induced inflow velocity (Equation 3.1, thesis p.52)
* $[v_E^I]^E$ is the relative wing in the propeller frame.
The origin of the vehicle is unclear, but:


## Code parameters

* camelCase unless stated otherwise
* Underscores for frames or for subscripts


In [1]:
%matplotlib inline
import numpy as np 
from scipy.spatial.transform import Rotation as R

In [90]:
bodyEulerAngles_w = np.pi/180*np.array([0,0,0])
rot_flyer2world = R.from_euler('xyz', bodyEulerAngles_w).as_dcm() # TODO: check this
totalNumBlades = 2
bladeRadius = 0.1
bladeIndex = 1
vel_cg2w_w = np.array([10, 0, 0])
angvel_b2w_w = np.array([0.0, 0.0, 50.0])
inflowVelocity = 2 # TODO nu
height_element2ref = 0.1 # TODO
prop_angvel = 10

def getRelativeWind(bladeIndex, totalNumBlades, bladeRadius, vel_cg2w_w, 
                    rot_flyer2world, angvel_b2w_w, inflowVelocity, height_element2ref,
                    prop_angvel):
    """
    _e is prop element frame
    _f is flyer frame
    _w is world/inertial frame
    TODO: inflow
    """

    # Inflow velocity is only coming from the top of the disc, so 
    # it is always in the positive z direction 
    vel_inducedInflow_e = np.array([0, 0, inflowVelocity])
    
    
    # Velocity of the flyer from the center of gravity moving forward
    # which is simply the negative of the cg velocity in the flyer frame 
    rotation_flyer2world = R.from_dcm(rot_flyer2world)
    euler_flyer2world = rotation_flyer2world.as_euler('xyz')
    # Get yaw from euler angles of flyer from world
    yaw = euler_flyer2world[2] # 3rd angle is yaw 
    propellor_angle = bladeIndex/totalNumBlades*2*np.pi + yaw
    
    rot_flyer2prop = R.from_euler('z', propellor_angle).as_dcm()
    
    rot_world2flyer = np.transpose(rot_flyer2world)
    vel_flyer2world_f = rot_world2flyer.dot(vel_cg2w_w)
    vel_flyer2world_e = rot_flyer2prop.dot(vel_flyer2world_f)

    
    
    # Velocity from rotational component
    # First calculate wxr=v in the flyer frame 
    # Then rotate v to the propellor frame 
    
    angvel_b2w_f = rot_world2flyer.dot(angvel_b2w_w).dot(rot_flyer2world)
    
    pos_x_blade = bladeRadius*np.sin(propellor_angle)
    pos_y_blade = bladeRadius*np.cos(propellor_angle)
    # I think sin and cos should be switched, but seems to work
    # like this. TODO: figure out. 
    r_cg2propElement_f = np.array([pos_x_blade, pos_y_blade, height_element2ref])
    
    
    angvel_spin_f = np.array([0, 0, prop_angvel])
    # Total angular velocity of the propellor is body + prop
    angvel_prop_f = angvel_b2w_f + angvel_spin_f
    vel_spin_f = np.cross(angvel_prop_f, r_cg2propElement_f)
    vel_spin_e = rot_flyer2prop.dot(vel_spin_f)
    
    # Sum all forces together
    relativeWind_e = -(vel_flyer2world_e + vel_inducedInflow_e + vel_spin_e)
    relWindAngleGamma = np.arctan2(relativeWind_e[2], relativeWind_e[0])

    return (relativeWind_e, relWindAngleGamma)


# Using XYZ euler angles such that the last angle Z is the correct psi angle

(relWind, gamma) = getRelativeWind(bladeIndex, totalNumBlades, bladeRadius, vel_cg2w_w, 
                    rot_flyer2world, angvel_b2w_w, inflowVelocity, height_element2ref, 
                                    prop_angvel)

print('gamma', gamma*180/np.pi)
print('relwind', relWind)

propellor angle 3.141592653589793
gamma -7.125016348901798
relwind [ 1.6000000e+01 -1.2246468e-15 -2.0000000e+00]


In [65]:

def piccoliGetRelWind(vel_cg2w_w, bodyEulerAngles_w, bladeIndex, 
                      totalNumBlades, rot_flyer2world, inflowVelocity, bladeRadius,
                      height_element2ref):
    #calculate relative wind
    rot_world2flyer = np.transpose(rot_flyer2world)
    vel_cg_f = rot_world2flyer.dot(vel_cg2w_w)
    yaw = bodyEulerAngles_w[2] # 3rd angle is yaw 
    propellor_angle = bladeIndex/totalNumBlades*2*np.pi + yaw
    height_prop_below_flyer_cg = 0.1 # TODO
    rot_flyer2prop_e = R.from_euler('z', propellor_angle).as_dcm()
    angvel_b2w_f = rot_world2flyer.dot(angvel_b2w_w).dot(rot_flyer2world)
    yawRate = angvel_b2w_f[2]
    angVel_e2w_f = rot_world2flyer.dot(angvel_b2w_w + [0, 0, yawRate])
    
    relativeWind_zcomponent_e = (vel_cg_f[0] + angVel_e2w_f[1]*height_element2ref - 
                                bladeRadius*angVel_e2w_f[2])
    relativeWind_xcomponent_e = (vel_cg_f[2] + bladeRadius*angVel_e2w_f[0] - 
                                inflowVelocity)
    relWindAngleGamma = np.arctan2(relativeWind_xcomponent_e,relativeWind_xcomponent_e)
    return (relativeWind_xcomponent_e, relativeWind_zcomponent_e, relWindAngleGamma)

piccoliGetRelWind(vel_cg2w_w, bodyEulerAngles_w, bladeIndex, 
                      totalNumBlades, rot_flyer2world, inflowVelocity, bladeRadius,
                      height_element2ref)

(1.7768573916115207, 12.182258696282728, 0.7853981633974483)

In [ ]:
# Blade equations 
def getBladeForceMoment(rot_blade2world_w, v_wind_w):
    """Returns force and moment vectors of a single blade in the world frame
        Arguements:
            rot_blade2world_w: rotation matrix from the blade to the world frame in world frame
            v_wind_w : velocity of wind in the world frame 
    """
    
    
    vel_wind_flyer = np.atan2()
    
    
    